In [1]:


%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import itertools
from sklearn.model_selection import cross_val_score, ShuffleSplit, RandomizedSearchCV, train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import make_scorer, confusion_matrix,SCORERS
from sklearn.dummy import DummyRegressor, DummyClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split



In [11]:
df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab3/bank-additional-full.csv", delimiter = ";")

df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [12]:
df.shape

(41188, 21)

In [17]:
df.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [7]:
df[['y']] = df[['y']].apply(LabelEncoder().fit_transform)
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0


In [8]:
df_dummies = pd.get_dummies(df)
df_dummies.head()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,56,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,0,0,1,0,0,0,0,1,0
1,57,149,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,0,0,1,0,0,0,0,1,0
2,37,226,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,0,0,1,0,0,0,0,1,0
3,40,151,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,0,0,1,0,0,0,0,1,0
4,56,307,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,0,0,1,0,0,0,0,1,0


In [9]:
df_dummies.drop(['duration'],inplace=True,axis=1)

In [10]:
df_dummies.isna().sum() # checking if there is any null values

age                     0
campaign                0
pdays                   0
previous                0
emp.var.rate            0
                       ..
day_of_week_tue         0
day_of_week_wed         0
poutcome_failure        0
poutcome_nonexistent    0
poutcome_success        0
Length: 63, dtype: int64

In [20]:


y = df_dummies['y']
X =df_dummies.drop(['y'],axis=1)



In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y,test_size=0.07,random_state=50)

In [25]:
X_test.shape

(2884, 62)

# Dummy Classifier

In [22]:
dummy_clf = DummyClassifier(strategy='stratified')
dummy_scores = cross_val_score(dummy_clf,X_train, y_train, cv=10, scoring=make_scorer(f1_score))  # cross-validation
print("F1 Score is:" , (dummy_scores.mean()))

F1 Score is: 0.11645865776374528


In [23]:
import sklearn.metrics as m
clf = RandomForestClassifier(n_estimators=100, max_depth=6,random_state=60)
clf.fit(X_train, y_train)
score = m.classification_report(y_test, clf.predict(X_test))
#scores = cross_val_score(clf,X_train, y_train, cv=10, scoring=make_scorer(f1_score))  # cross-validation
score

'              precision    recall  f1-score   support\n\n           0       0.91      0.99      0.95      7310\n           1       0.69      0.20      0.31       928\n\n    accuracy                           0.90      8238\n   macro avg       0.80      0.59      0.63      8238\nweighted avg       0.88      0.90      0.87      8238\n'


# Top 3 Features are :

# Feature ranking:

    # euribor3m
    # nr.employed
    # pdays



In [26]:
rfc = RandomForestClassifier(n_estimators=100, max_depth=10,random_state=60)
rfc.fit(X_train, y_train)

importances = rfc.feature_importances_
std = np.std([tree.feature_importances_ for tree in rfc.estimators_],axis=0)
indices = np.argsort(importances)[::-1]
print(indices)
features = X.columns
# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. %s (%f)" % (f + 1, features[indices[f]],  importances[indices[f]]))

[ 7  8 61  2  4  6  5  0  3  1 42 49 60 52 59 43 50 44 55 48 56 31 10 33
 53 23 38 22 14 36 57 18 30 58 39 54  9 41 28 17 34 27 21 47 32 16 13 25
 51 15 26 46 45 37 19 40 12 20 11 24 29 35]
Feature ranking:
1. euribor3m (0.148873)
2. nr.employed (0.127016)
3. poutcome_success (0.090293)
4. pdays (0.080938)
5. emp.var.rate (0.077486)
6. cons.conf.idx (0.065699)
7. cons.price.idx (0.055332)
8. age (0.045259)
9. previous (0.033061)
10. campaign (0.017368)
11. contact_cellular (0.016358)
12. month_mar (0.015451)
13. poutcome_nonexistent (0.014476)
14. month_oct (0.014365)
15. poutcome_failure (0.012378)
16. contact_telephone (0.011877)
17. month_may (0.011095)
18. month_apr (0.008402)
19. day_of_week_mon (0.008157)
20. month_jun (0.007739)
21. day_of_week_thu (0.005571)
22. education_university.degree (0.005529)
23. job_blue-collar (0.005486)
24. default_no (0.005405)
25. month_sep (0.004877)
26. marital_single (0.004863)
27. housing_yes (0.004783)
28. marital_married (0.004732)
29. job_re

In [27]:
rfc.score(X_test, y_test) 

0.8997919556171984

In [ ]:
# Compute confusion matrix
y_pred = rfc.predict(X_test)
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=range(len(set(y))), normalize=True,
                      title='Confusion matrix')

#plt.savefig("confusion.pdf", bbox_inches='tight')
